In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install crossfit --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.6/178.6 kB 13.5 MB/s eta 0:00:00


In [5]:
import dask_cudf
import time
import pandas as pd
import cudf
from dask.distributed import Client, LocalCluster
import pandas as pd
start = time.time()
# Initialize Dask LocalCluster
cluster = LocalCluster(n_workers=1, threads_per_worker=2)
client = Client(cluster)
print(f"Dask dashboard: {client.dashboard_link}")

INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:34027
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:36441'
INFO:distributed.scheduler:Register worker addr: tcp://127.0.0.1:41237 name: 0
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:41237
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:49494
INFO:distributed.scheduler:Receive client connection: Client-dcedd03e-54cf-11f0-837d-0242ac1c000c
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:49502


Dask dashboard: http://127.0.0.1:8787/status


In [6]:
## in this cell block we are adding an if column named "id" to the dataset, this is important and mandatory to do
## as at later stages, we use this id to uniquely identify documents.

from add_id import add_id_column
input_path = "/content/train-00000-of-00001.parquet"
data_id = "my_data_with_id.parquet"
input_id = add_id_column(input_path, data_id)


df_with_id = pd.read_parquet(input_id)
df_with_id.head()

Added 'id' column to my_data_with_id.parquet


,id,code
0,0,'use strict';\n\nvar clear = require(...
1,1,'use strict';\n\nconst TYPE = Symbol.for('type...
2,2,package sodium\n\n// #cgo pkg-config: libsodiu...
3,3,\nfunction collectWithWildcard(test) {\n\ttest...
4,4,<?php\n\ninterface Container {\n /**\n ...


In [7]:
df_with_id.shape

(5000, 2)

In [8]:
## in this cell block we just create embedding for the column(text_column) using a embedding
## model, it saves output data as parquet So final table has id and embedding columns

from embedding import ParquetEmbeddingCreator
start_load_embedding_model = time.time()
output_path = "embeddings.parquet"
creator = ParquetEmbeddingCreator(
    model_name_or_path="sentence-transformers/all-MiniLM-L6-v2", #embedding model to be used.
    id_column="id",      # Your ID column name
    text_column="code",        # Your text column name
    embedding_column="embeddings",
    output_dir=output_path
)
time_load_embedding_model = time.time() - start_load_embedding_model
result_df = creator(input_id) # Also saves the embedding file in parquet format at directory specified at output_dir

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Fitting memory estimate curve for model: sentence-transformers/all-MiniLM-L6-v2


Batch size: 100%|██████████| 8/8 [00:48<00:00,  6.05s/it]


In [9]:
# verifying that embeddings are created properly and the size of the table.
df = pd.read_parquet(output_path)
df.head()

,id,embeddings
0,0,"[-0.03688268, 0.01722964, 0.014153543, -0.0107..."
1,1,"[-0.010918071, 0.06880616, -0.02336217, 0.0540..."
2,2,"[-0.07070189, 0.0055253967, -0.029556742, -0.0..."
3,3,"[-0.08766589, 0.017386748, -0.0018631271, 0.01..."
4,4,"[0.03449858, 0.06963933, 0.03905583, 0.0027986..."


In [10]:
df.shape

(5000, 2)

In [11]:
# in this step we apply k-means, to cluster entire dataset into n_clusters
# further we will check only documents withing cluster as potential duplicates.

from clustering import ParquetEmbeddingClusterer

clusterer = ParquetEmbeddingClusterer(
        n_clusters=50,
        max_iter=10,
        clustering_output_dir="clustering_output",
        id_column="id",
        embedding_column="embeddings",
        keep_all_columns=False
    )
result = clusterer(output_path)

In [12]:
# in the table we can also see the L2 and cosine distance to the particular centroid in which
# it is mapped.
first_partition = result.get_partition(0).compute()
first_partition.head()

,id,embeddings,l2_dist_to_centroid,cosine_dist_to_centroid
0,17,"[-0.039881885915448695, 0.12994559666953232, -...",0.925009,0.839316
1,213,"[0.0018064314519195576, -0.029548872336090333,...",0.763266,0.702783
2,216,"[-0.061975960830168694, -0.040372629462373806,...",0.916509,0.831490
3,310,"[-0.010521605396688557, 0.03472509099304494, -...",0.901843,0.818156
4,313,"[-0.03252442633648523, 0.06066072881433729, -0...",0.939404,0.852736


In [13]:
# cluster statistics for some clusters.
for i in [0,1,12,24,37]:
    try:
        cluster_path = f"clustering_output/embs_by_nearest_center/nearest_cent={i}/part.0.parquet"
        df = cudf.read_parquet(cluster_path)
        print(f"Cluster {i}:")
        print(f"  Shape: {df.shape}")
        print(f"  Columns: {list(df.columns)}")
        if len(df) > 0:
            print(f"  First row ID: {df.iloc[0]['id'] if 'id' in df.columns else 'NO ID COLUMN'}")
            print(f"  Has embeddings: {'embeddings' in df.columns}")
            print(f"  Has cosine_dist_to_cent: {'cosine_dist_to_cent' in df.columns}")
            if 'embeddings' in df.columns:
                print(f"  Embedding type: {type(df['embeddings'].iloc[0])}")
        print("---")
    except Exception as e:
        print(f"Error reading cluster {i}: {e}")
        print("---")

Cluster 0:
  Shape: (93, 5)
  Columns: ['id', 'embeddings', 'l2_dist_to_centroid', 'cosine_dist_to_centroid', 'nearest_cent']
  First row ID: 17
  Has embeddings: True
  Has cosine_dist_to_cent: False
  Embedding type: <class 'list'>
---
Cluster 1:
  Shape: (106, 5)
  Columns: ['id', 'embeddings', 'l2_dist_to_centroid', 'cosine_dist_to_centroid', 'nearest_cent']
  First row ID: 59
  Has embeddings: True
  Has cosine_dist_to_cent: False
  Embedding type: <class 'list'>
---
Cluster 12:
  Shape: (57, 5)
  Columns: ['id', 'embeddings', 'l2_dist_to_centroid', 'cosine_dist_to_centroid', 'nearest_cent']
  First row ID: 74
  Has embeddings: True
  Has cosine_dist_to_cent: False
  Embedding type: <class 'list'>
---
Cluster 24:
  Shape: (42, 5)
  Columns: ['id', 'embeddings', 'l2_dist_to_centroid', 'cosine_dist_to_centroid', 'nearest_cent']
  First row ID: 22
  Has embeddings: True
  Has cosine_dist_to_cent: False
  Embedding type: <class 'list'>
---
Cluster 37:
  Shape: (46, 5)
  Columns: ['id'

In [14]:
# loading cluster 9 data
df = pd.read_parquet("/content/clustering_output/embs_by_nearest_center/nearest_cent=9/part.0.parquet")
df.head()

,id,embeddings,l2_dist_to_centroid,cosine_dist_to_centroid
0,50,"[-0.04503113274387922, 0.07435173327440658, -0...",0.770826,0.623813
1,86,"[0.027017301443950262, 0.02267774794859521, -0...",0.792440,0.640707
2,100,"[-0.013128006204540165, -0.055196279079800244,...",0.778570,0.629812
3,115,"[-0.03824186480136379, -0.01161733409929698, -...",0.819609,0.662606
4,128,"[-0.03838233870554475, 0.062386767335556814, 0...",0.757059,0.613295


In [15]:
df.shape # basically means in cluster 9 we have 135 documents, with their id's and embeddings as given

(135, 4)

In [16]:
## this is final step in sematic deduplication, wherein for each cluster we calculate
## the cosine score or the L2 score (sim_metric), and according to the threshold we identify them as
## duplicates. For eg: say after clustering in cluster x we had 100 documents, and after this code block
## is run, cluster x has 80 documents, means 80 were considered duplicates and should be removed, with other 20
## not classfied as duplicates.

from semdedup import GPUSemanticDeduplicator

deduplicator = GPUSemanticDeduplicator(
        clustering_results_dir="clustering_output",
        id_column="id",
        embedding_column="embeddings",
        which_to_keep="hard",
        sim_metric="cosine",
        output_dir="semdedup_results",
        batched_cosine_similarity=1024,
    )
# Step 1: Compute semantic match dataframes
deduplicator.compute_semantic_match_dfs()

# Step 2: Extract documents to remove at given epsilon
eps_to_extract = 0.3 # cosine similarity above 0.7 is considered similar documents
documents_to_remove = deduplicator.extract_dedup_data(eps_to_extract)

In [17]:
# final number of documents to be removed can be seen here with their id's
df = pd.read_parquet("/content/semdedup_results/unique_ids_0.3.parquet")
df.shape

(1005, 3)

In [18]:
df.head() # here cluster denotes the cluster to which it belongs
# very important these are document id's to be removed, so final deduplicated dataset would be
# total number of documents - these documents to be removed.

,id,cosine_dist_to_centroid,cluster
0,213,0.702783,0
1,317,0.704896,0
2,710,0.702998,0
3,2834,0.701405,0
4,3235,0.701201,0


In [19]:
#closing the dask cluster
cluster.close()

INFO:distributed.scheduler:Retire worker addresses (stimulus_id='retire-workers-1751191383.62373') (0,)
INFO:distributed.nanny:Closing Nanny at 'tcp://127.0.0.1:36441'. Reason: nanny-close
INFO:distributed.nanny:Nanny asking worker to close. Reason: nanny-close
INFO:distributed.core:Received 'close-stream' from tcp://127.0.0.1:49494; closing.
INFO:distributed.scheduler:Remove worker addr: tcp://127.0.0.1:41237 name: 0 (stimulus_id='handle-worker-cleanup-1751191383.636803')
INFO:distributed.scheduler:Lost all workers
INFO:distributed.nanny:Worker process 6340 was killed by signal 9
INFO:distributed.nanny:Nanny at 'tcp://127.0.0.1:36441' closed.
INFO:distributed.scheduler:Closing scheduler. Reason: unknown
INFO:distributed.scheduler:Scheduler closing all comms


In [20]:
#closing the dask client.
client.close()

In [21]:
total_time = time.time() - start

In [22]:
## when loading the embedding model, it takes significant amount of time, so we need to consider that
## it has no implication on time taken to semantically dedup the data
print(f"Total time taken: {total_time} seconds")
print(f"Time to load embedding model: {time_load_embedding_model} seconds")
print(f"Time to deduplication: {total_time - time_load_embedding_model} seconds")

Total time taken: 173.35539484024048 seconds
Time to load embedding model: 71.79692435264587 seconds
Time to deduplication: 101.5584704875946 seconds
